In [ ]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ---- CONFIG ----
folder_path = 'robustness_results_other_aggregation'
dike_ids = ['A.1', 'A.2', 'A.3', 'A.4', 'A.5']
alpha_levels = [0.5, 0.3, 0.2, 0.05]
colors = sns.color_palette("Set2", len(dike_ids))  # 5 distinct colors

# ---- FUNCTIONS ----
def plot_deaths_vs_costs(df, policy_name):
    plt.figure(figsize=(8, 6))
    for i, dike in enumerate(dike_ids):
        deaths_col = f'{dike}_Expected Number of Deaths'
        cost_col = f'{dike} Total Costs'
        if deaths_col in df.columns and cost_col in df.columns:
            plt.scatter(df[cost_col], df[deaths_col],
                        label=dike, color=colors[i], alpha=0.5, s=12)

    plt.xlabel('Total Costs')
    plt.ylabel('Expected Number of Deaths')
    plt.title(f'Deaths vs Costs per Dike Ring\n{policy_name}')
    plt.legend(title='Dike Ring')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def compute_summary_with_regret(df, policy_name):
    summary = []
    df['scenario'] = df.index  # Ensure scenario index exists

    for dike in dike_ids:
        deaths_col = f'{dike}_Expected Number of Deaths'
        cost_col = f'{dike} Total Costs'

        if deaths_col in df.columns and cost_col in df.columns:
            # Regret calculation per scenario
            deaths_regret = df[deaths_col] - df[deaths_col].min()
            cost_regret = df[cost_col] - df[cost_col].min()

            summary.append({
                'Policy': policy_name,
                'Dike Ring': dike,
                'Exp. Deaths (Mean)': df[deaths_col].mean(),
                'Max Regret (Deaths)': deaths_regret.max(),
                'Investment Cost (Mean)': df[cost_col].mean(),
                'Max Regret (Invest.)': cost_regret.max(),
            })

    return pd.DataFrame(summary)

# ---- MAIN LOOP ----
all_summaries = []

for filename in os.listdir(folder_path):
    if '_outcomes_' in filename and filename.endswith('.csv'):
        filepath = os.path.join(folder_path, filename)
        df = pd.read_csv(filepath)

        # Identify policy
        match = re.search(r'policy_(\d+)_s(\d+)', filename)
        if match:
            policy_id = match.group(1)
            seed_id = match.group(2)
            policy_name = f'Policy {policy_id} (Seed {seed_id})'

            # 2. Compute and print summary
            summary_df = compute_summary_with_regret(df, policy_name)
            summary_df.to_string
            all_summaries.append(summary_df)

# Combine all into one table
final_summary_df = pd.concat(all_summaries, ignore_index=True)

# Show full summary (optional)
print("\n✅ Combined Summary Table Across All Policies:")
final_summary_df.head(100)
final_summary_df.to_csv(os.path.join(folder_path, 'combined_outcomes_summary.csv'), index=False)

In [ ]:
import numpy as np

# Define the exact desired policy order
desired_order = [
    "Policy 6 (Seed 20)",
    "Policy 8 (Seed 20)",
    "Policy 7 (Seed 21)",
    "Policy 8 (Seed 21)"
]

# Ensure Policy column is treated as categorical for consistent ordering
final_summary_df['Policy'] = pd.Categorical(final_summary_df['Policy'], categories=desired_order, ordered=True)

# Pivot data: rows are policies, columns are dike rings
deaths_pivot = final_summary_df.pivot(index='Policy', columns='Dike Ring', values='Exp. Deaths (Mean)')
costs_pivot = final_summary_df.pivot(index='Policy', columns='Dike Ring', values='Investment Cost (Mean)')

# Set color palette for consistent dike coloring
dike_colors = dict(zip(dike_ids, colors))

# Create side-by-side subplots with extra spacing between them
fig, axs = plt.subplots(1, 2, figsize=(18, 6), gridspec_kw={'wspace': 0.4})  # increased wspace

# Bar width (smaller)
bar_width = 0.6
x = np.arange(len(deaths_pivot))

# Plot 1: Stacked Barplot for Expected Deaths
bottom = np.zeros(len(deaths_pivot))
for dike in dike_ids:
    axs[0].bar(x, deaths_pivot[dike], bottom=bottom,
               label=dike, color=dike_colors[dike], width=bar_width)
    bottom += deaths_pivot[dike].fillna(0)

axs[0].set_ylabel("Expected Number of Deaths")
axs[0].set_title("Expected Deaths per Policy")
axs[0].set_xticks(x)
axs[0].set_xticklabels(deaths_pivot.index, rotation=45)
axs[0].grid(True, axis='y')

# Plot 2: Stacked Barplot for Investment Costs
bottom = np.zeros(len(costs_pivot))
for dike in dike_ids:
    axs[1].bar(x, costs_pivot[dike], bottom=bottom,
               label=dike, color=dike_colors[dike], width=bar_width)
    bottom += costs_pivot[dike].fillna(0)

axs[1].set_ylabel("Investment Costs")
axs[1].set_title("Investment Costs per Policy")
axs[1].set_xticks(x)
axs[1].set_xticklabels(costs_pivot.index, rotation=45)
axs[1].grid(True, axis='y')

# Common legend above
handles, labels = axs[1].get_legend_handles_labels()
fig.legend(handles, labels, title='Dike Ring', loc='upper center', ncol=len(dike_ids), bbox_to_anchor=(0.5, 1.12))

plt.tight_layout(rect=[0, 0, 1, 1.05])  # Leave space for the legend
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Desired policy order
desired_order = [
    "Policy 6 (Seed 20)",
    "Policy 8 (Seed 20)",
    "Policy 7 (Seed 21)",
    "Policy 8 (Seed 21)"
]
final_summary_df['Policy'] = pd.Categorical(final_summary_df['Policy'], categories=desired_order, ordered=True)

# Pivot data for means and regrets
deaths_mean = final_summary_df.pivot(index='Policy', columns='Dike Ring', values='Exp. Deaths (Mean)')
deaths_regret = final_summary_df.pivot(index='Policy', columns='Dike Ring', values='Max Regret (Deaths)')

costs_mean = final_summary_df.pivot(index='Policy', columns='Dike Ring', values='Investment Cost (Mean)')
costs_regret = final_summary_df.pivot(index='Policy', columns='Dike Ring', values='Max Regret (Invest.)')

# Colors for dike rings
colors = [
    '#b44447',  # muted strong red
    '#3f4f99',  # softer blue
    '#4a7c42',  # medium muted green
    '#c88a3d',  # warm mustard
    '#6f5a9c'   # dusty purple
]
dike_colors = dict(zip(dike_ids, colors))

# Setup figure and axes (2 rows x 2 columns)
fig, axs = plt.subplots(2, 2, figsize=(20, 12), gridspec_kw={'hspace': 0.5, 'wspace': 0.3})

x = np.arange(len(deaths_mean))
bar_width = 0.7

# Plot function for stacked bars
def plot_stacked_bar(ax, data, ylabel, title):
    bottom = np.zeros(len(data))
    for dike in dike_ids:
        ax.bar(x, data[dike], bottom=bottom, label=dike, color=dike_colors[dike], width=bar_width)
        bottom += data[dike].fillna(0)
    ax.set_xticks(x)
    ax.set_xticklabels(data.index, rotation=45, ha='right')
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.grid(True, axis='y')

# Top-left: Mean Deaths
plot_stacked_bar(axs[0,0], deaths_mean, "Expected Number of Deaths", "Mean Expected Deaths per Policy")

# Top-right: Mean Investment Costs
plot_stacked_bar(axs[0,1], costs_mean, "Total  Costs", "Total Costs per Policy")

# Bottom-left: Max Regret Deaths
plot_stacked_bar(axs[1,0], deaths_regret, "Max Regret (Deaths)", "Max Regret of Deaths per Policy")

# Bottom-right: Max Regret Investment Costs
plot_stacked_bar(axs[1,1], costs_regret, "Max Regret (Total Costs)", "Max Regret of Total Costs per Policy")

# Main Title
fig.suptitle("Expected Values and Maximum Regret per Policy and Dike Ring",
             fontsize=18, fontweight='bold', y=1.06)

# Legend
handles, labels = axs[0, 1].get_legend_handles_labels()
fig.legend(handles, labels,
           title='Dike Ring', title_fontsize=15, fontsize=13,
           loc='upper center', ncol=len(dike_ids), bbox_to_anchor=(0.5, 1.015))

# Adjust layout (room for legend + title)
plt.tight_layout(rect=[0, 0.05, 1, 0.95])

output_folder = 'plots'
os.makedirs(output_folder, exist_ok=True)
save_path = os.path.join(output_folder, 'top_policies_per_dike_ring.png')
plt.savefig(save_path, facecolor='white', edgecolor='white', bbox_inches='tight', dpi=300)

plt.show()
